In [ ]:
import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")
model = load_model("mask.h5")

In [ ]:
video_capture = cv2.VideoCapture(0)
while True:
    try:
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray,
                                             scaleFactor=1.1,
                                             minNeighbors=5,
                                             minSize=(60, 60),
                                             flags=cv2.CASCADE_SCALE_IMAGE)
        faces_list = []
        preds = []
        for (x, y, w, h) in faces:
            face_frame = frame[y:y + h, x:x + w]
            face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
            face_frame = cv2.resize(face_frame, (224, 224))
            face_frame = img_to_array(face_frame)
            face_frame = np.expand_dims(face_frame, axis=0)
            face_frame = preprocess_input(face_frame)
            faces_list.append(face_frame)
            if len(faces_list) > 0:
                preds = model.predict(faces_list)
            for pred in preds:
                (mask, withoutMask) = pred
            label = "OLA!!!" if mask > withoutMask else "USE MASCARA"
            color = (0, 255, 0) if label == "SEJA BEM VINDO" else (255, 255, 255)
            # label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
            label = "{}".format(label)
            cv2.putText(frame, label, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.55, color, 2)

            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

            # Display the resulting frame
        cv2.putText(frame, ' LabFront - detector de mascara', (0, 450),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.imshow('Video', frame)
        key = cv2.waitKey(10)

        if key == 27:
            break
    except:
        pass
video_capture.release()
cv2.destroyAllWindows()

KeyboardInterrupt: ignored